## Test Data 로 정확도 / Confusion Matrix 확인

### LoRA STF model test code

In [13]:
import re
import os
import json
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer
from peft import PeftModel

# 로깅 설정
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [14]:

def load_finetuned_model(model_path, device="cuda"):
    """파인튜닝된 모델 로드"""
    logger.info(f"Loading fine-tuned model from {model_path}")
    
    # 프로세서와 토크나이저 로드
    processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    
    # 모델 로드 (LoRA 어댑터 포함)
    base_model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Vision-Instruct-3B"
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        trust_remote_code=True,
        revision="main",
        device_map={"": device},
        torch_dtype=torch.float16
    )
    
    # LoRA 어댑터 로드
    model = PeftModel.from_pretrained(model, model_path)
    model = model.merge_and_unload()  # LoRA를 베이스 모델에 병합
    model.eval()
    
    return model, processor, tokenizer


In [15]:
def load_base_model(device="cuda"):
    """파인튜닝되지 않은 베이스 모델 로드"""
    model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Vision-Instruct-3B"
    revision = "v0.1.0"  # 원하는 revision으로 고정

    logger.info(f"Loading base model: {model_name} ({revision})")
    
    processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True, revision=revision)
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, revision=revision)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        revision=revision,
        device_map={"": device},
        torch_dtype=torch.float16
    )
    model.eval()

    return model, processor, tokenizer

In [16]:
def move_to_device(inputs, device):
    for k, v in inputs.items():
        if isinstance(v, torch.Tensor):
            inputs[k] = v.to(device)
        elif isinstance(v, list):
            if all(isinstance(i, torch.Tensor) for i in v):
                inputs[k] = [i.to(device) for i in v]
        elif isinstance(v, dict):
            for sub_k, sub_v in v.items():
                if isinstance(sub_v, torch.Tensor):
                    v[sub_k] = sub_v.to(device)
    return inputs

In [17]:
def test_model(model, processor, tokenizer, test_data_path, output_dir, model_path, device="cuda"):
    """테스트 데이터셋으로 모델 평가"""
    
    # 테스트 데이터 로드
    logger.info(f"Loading test data from {test_data_path}")
    with open(test_data_path, 'r', encoding='utf-8') as f:
        test_data = [json.loads(line) for line in f]
    
    # 감정 라벨 정의 (system prompt에 정의된 순서대로)
    emotions = ['행복', '슬픔', '분노', '공포', '놀람', '중립']
    
    # 상세 로그 파일 생성
    log_filename = f"{output_dir}/detailed_predictions_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    log_file = open(log_filename, 'w', encoding='utf-8')
    log_file.write("=== Emotion Classification Detailed Test Log ===\n")
    log_file.write(f"Test started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_file.write(f"Model path: {model_path}\n")
    log_file.write(f"Total test samples: {len(test_data)}\n")
    log_file.write("="*60 + "\n\n")
    
    predictions = []
    true_labels = []
    correct_count = 0
    
    logger.info("Starting evaluation...")
    for idx, sample in enumerate(tqdm(test_data, desc="Evaluating")):
        # 데이터 추출
        image_path = None
        user_text = ""
        true_emotion = ""

        for msg in sample['chat']:
            content = msg['content']
            if isinstance(content, dict):
                if content.get("type") == "image":
                    image_path = content.get("image")
                elif content.get("type") == "text" and msg["role"] == "user":
                    user_text = content.get("text")
                elif content.get("type") == "text" and msg["role"] == "assistant":
                    true_emotion = content.get("text")


        # 프롬프트 생성
        conversations = [
            {
                "role": "system",
                "content": "제시된 이미지와 텍스트를 함께 고려하여 인물의 감정을 판단하십시오. 응답은 반드시 아래 여섯 가지 중 하나여야 합니다 : 행복, 슬픔, 분노, 공포, 놀람, 중립."
            },
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image_path},  # ✅ 반드시 포함!
                    {"type": "text", "text": user_text}
                ]
            }
        ]

        # 1. 프롬프트 생성
        text = processor.apply_chat_template(
            conversations,
            tokenize=False,
            add_generation_prompt=True
        )

        # 2. 이미지 로드
        if not image_path or not os.path.exists(image_path):
            print(f"[{idx+1}] ⚠️ 이미지 경로 없음 또는 존재하지 않음: {image_path}")
            log_file.write(f"[Sample {idx+1}] WARNING: Missing or invalid image path: {image_path}\n")
            continue

        try:
            with Image.open(image_path) as img:
                img.verify()  # 이미지 포맷/헤더 유효성만 검사 (데이터는 읽지 않음)

            with Image.open(image_path) as img:
                img.load()  # 실제 데이터 로드 (깨진 이미지 확인용)
                image = img.convert("RGB").copy()  # 복사해두고 파일 핸들 닫기
        except Exception as e:
            print(f"[{idx+1}] ❌ 이미지 로딩 오류: {e}")
            log_file.write(f"[Sample {idx+1}] ERROR during image load: {e}\n\n")
            continue

        # print(f"[{idx+1}] 📷 image type: {type(image)} | size: {image.size}")
        
        # 3. 텍스트 + 이미지 통합 전처리 (권장 방식)
        try:
            inputs = processor(
                text=[text],
                images=[image],
                return_tensors="pt",
                padding=True
            )

            # ✅ pixel_values 보완
            if "pixel_values_images" in inputs:
                try:
                    pixel_tensor = inputs["pixel_values_images"][0][0]
                    inputs["pixel_values"] = pixel_tensor.to(device)

                    # ✅ 리스트 내부 텐서도 이동
                    inputs["pixel_values_images"] = [
                        [t.to(device) if isinstance(t, torch.Tensor) else t for t in sublist]
                        for sublist in inputs["pixel_values_images"]
                    ]

                except Exception as e:
                    raise RuntimeError(f"❌ 'pixel_values_images' 처리 중 오류: {e}")
            else:
                raise KeyError("❌ 'pixel_values_images'가 processor 결과에 존재하지 않습니다.")

            # ✅ 나머지 텐서 to(device)
            inputs = {
                k: v.to(device) if isinstance(v, torch.Tensor) else v
                for k, v in inputs.items()
            }
            
            
        except Exception as e:
            print(f"[{idx+1}] ❌ processor 오류: {e}")
            log_file.write(f"[Sample {idx+1}] ERROR during processor call: {e}\n\n")
            continue
        
        # 4. 추론
        try:
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=10,
                    do_sample=False,
                    pad_token_id=tokenizer.eos_token_id
                    # temperature=0.1,
                )
        except Exception as e:
            print(f"[{idx+1}] ❌ generate 오류: {e}")
            log_file.write(f"[{idx+1}] ERROR: Generate failed: {e}\n\n")
            continue
        
        # 결과 디코딩 (generated_text 중 감정 라벨만 추출)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # 감정 라벨만 정제해서 추출
        emotion_labels = ["행복", "슬픔", "분노", "공포", "놀람", "중립"]
        pattern = "|".join(emotion_labels)
        match = re.search(pattern, generated_text)

        predicted_emotion = match.group(0) if match else "unknown"
        
        # 예측 결과 정리
        predictions.append(predicted_emotion)
        true_labels.append(true_emotion)
        is_correct = predicted_emotion == true_emotion
        if is_correct:
            correct_count += 1
        
        log_file.write(f"[{idx+1}]\nImage: {os.path.basename(image_path)}\n")
        log_file.write(f"Text: {user_text}\nTrue: {true_emotion} | Predicted: {predicted_emotion} | {'✓' if is_correct else '✗'}\n")
        log_file.write(f"Generated: {generated_text}\n{'-'*40}\n\n")

        if (idx + 1) % 50 == 0:
            acc = correct_count / (idx + 1)
            logger.info(f"✅ Progress: {idx+1}/{len(test_data)} | Accuracy: {acc:.4f}")

    final_accuracy = correct_count / len(predictions)
    log_file.write("="*60 + "\n")
    log_file.write(f"✅ Final accuracy: {final_accuracy:.4f} ({correct_count}/{len(predictions)})\n")
    log_file.write(f"🕒 Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_file.close()
    
    logger.info(f"Detailed log saved to: {log_filename}")
    
    return predictions, true_labels

In [18]:
def calculate_metrics(true_labels, predictions):
    """성능 메트릭 계산"""
    
    # 기본 메트릭
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted', zero_division=0)
    recall = recall_score(true_labels, predictions, average='weighted', zero_division=0)
    f1 = f1_score(true_labels, predictions, average='weighted', zero_division=0)
    
    # 클래스별 메트릭
    labels = sorted(set(true_labels + predictions))
    class_report = classification_report(
        true_labels, 
        predictions, 
        target_names=labels,
        output_dict=True,
        zero_division=0
    )
    
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'class_report': class_report
    }
    
    return metrics

In [19]:
def plot_confusion_matrix(true_labels, predictions, save_path="confusion_matrix.png"):
    """혼동 행렬 시각화"""
    
    # 라벨 정렬
    labels = sorted(set(true_labels + predictions))
    
    # 혼동 행렬 계산
    cm = confusion_matrix(true_labels, predictions, labels=labels)
    
    # 한글 폰트 설정 (필요시)
    plt.rcParams['font.family'] = 'DejaVu Sans'
    
    # 시각화
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        xticklabels=labels,
        yticklabels=labels
    )
    plt.title('Confusion Matrix - Emotion Classification', fontsize=16)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()
    
    # 정규화된 혼동 행렬
    with np.errstate(divide='ignore', invalid='ignore'):
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_normalized = np.nan_to_num(cm_normalized)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm_normalized, 
        annot=True, 
        fmt='.2f', 
        cmap='Blues',
        xticklabels=labels,
        yticklabels=labels
    )
    plt.title('Normalized Confusion Matrix - Emotion Classification', fontsize=16)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.tight_layout()
    plt.savefig(save_path.replace('.png', '_normalized.png'), dpi=300)
    plt.close()
    
    return cm


In [20]:
def save_results(predictions, true_labels, metrics, output_dir):
    """결과 저장"""
    
    # 예측 결과 저장
    results_df = pd.DataFrame({
        'true_label': true_labels,
        'predicted_label': predictions,
        'correct': [t == p for t, p in zip(true_labels, predictions)]
    })
    results_df.to_csv(f"{output_dir}/predictions.csv", index=False, encoding='utf-8')
    
    # 메트릭 저장
    with open(f"{output_dir}/metrics.json", 'w', encoding='utf-8') as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)
    
    # 요약 리포트 생성
    report = f"""
=== Emotion Classification Test Results ===

Overall Metrics:
- Accuracy: {metrics['accuracy']:.4f}
- Precision: {metrics['precision']:.4f}
- Recall: {metrics['recall']:.4f}
- F1 Score: {metrics['f1_score']:.4f}

Per-Class Performance:
"""
    
    for emotion, scores in metrics['class_report'].items():
        if emotion not in ['accuracy', 'macro avg', 'weighted avg']:
            report += f"\n{emotion}:"
            report += f"\n  - Precision: {scores['precision']:.4f}"
            report += f"\n  - Recall: {scores['recall']:.4f}"
            report += f"\n  - F1-Score: {scores['f1-score']:.4f}"
            report += f"\n  - Support: {scores['support']}"
    
    # 오분류 분석 추가
    report += "\n\n=== Error Analysis ===\n"
    error_matrix = {}
    for true, pred in zip(true_labels, predictions):
        if true != pred:
            key = f"{true} → {pred}"
            error_matrix[key] = error_matrix.get(key, 0) + 1
    
    report += "\nMost common misclassifications:\n"
    for error, count in sorted(error_matrix.items(), key=lambda x: x[1], reverse=True)[:10]:
        report += f"  {error}: {count} times\n"
    
    with open(f"{output_dir}/test_report.txt", 'w', encoding='utf-8') as f:
        f.write(report)
    
    logger.info(f"Results saved to {output_dir}")

In [21]:
# ✅ 디바이스 강제 이동 함수
def force_move_all(model, device):
    model.to(device)
    for name, module in model.named_children():
        try:
            module.to(device)
        except:
            pass

# ✅ 디바이스 확인용 (디버깅용이므로 추후 제거해도 됨)
def print_model_devices(model, device):
    print("📦 Model 서브모듈 디바이스 확인:")
    for name, module in model.named_modules():
        try:
            param = next(module.parameters())
            if param.device != torch.device(device):
                print(f"❌ {name}: {param.device} (Expected: {device})")
            else:
                print(f"✅ {name}: {param.device}")
        except StopIteration:
            print(f"⚠️  {name}: 파라미터 없음")

In [22]:

def main():
    # 설정
    model_path = "/home/jieun/workspace/clovax/finetune/output/finetuned_hcx_sft_masking"  # 학습된 모델 경로
    test_data_path = "/home/jieun/workspace/clovax/finetune/dataset/test_data/test_dataset.jsonl"  # 테스트 데이터 경로
    output_dir = "./test_results"
    device = "cuda:7" if torch.cuda.is_available() else "cpu"
    
    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    
    # 파인튜닝된 모델 로드
    model, processor, tokenizer = load_finetuned_model(model_path, device)
    
    print("🚀 모델 디바이스:", next(model.parameters()).device)
    
    # print_model_devices(model, device)
    # 💪 강제로 모두 이동
    force_move_all(model, device)

    # 🔁 이동 후 확인
    print("✅ 이동 후 상태:")
    # print_model_devices(model, device)
    
    for name, param in model.named_parameters():
        print(f" - {name}: {param.device}")
        break
    
    force_move_all(model, device)
    print_model_devices(model, device)
    
    # 파인튜닝된 모델로 테스트 실행
    predictions, true_labels = test_model(
        model, processor, tokenizer, test_data_path, output_dir, model_path, device
    )
    
    # 메트릭 계산
    metrics = calculate_metrics(true_labels, predictions)
    
    # 혼동 행렬 그리기
    cm = plot_confusion_matrix(
        true_labels, 
        predictions, 
        save_path=f"{output_dir}/confusion_matrix.png"
    )
    
    # 결과 저장
    save_results(predictions, true_labels, metrics, output_dir)
    
    # 결과 출력
    print("\n=== Test Results ===")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1_score']:.4f}")
    
    print("\n=== Per-Class Performance ===")
    emotions = ['행복', '슬픔', '분노', '공포', '놀람', '중립']
    for emotion in emotions:
        if emotion in metrics['class_report']:
            scores = metrics['class_report'][emotion]
            print(f"{emotion}: P={scores['precision']:.3f}, R={scores['recall']:.3f}, F1={scores['f1-score']:.3f}, Support={scores['support']}")

if __name__ == "__main__":
    main()

INFO:__main__:Loading fine-tuned model from /home/jieun/workspace/clovax/finetune/output/finetuned_hcx_sft_masking


Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.04s/it]
INFO:__main__:Loading test data from /home/jieun/workspace/clovax/finetune/dataset/test_data/test_dataset.jsonl
INFO:__main__:Starting evaluation...


🚀 모델 디바이스: cuda:7
✅ 이동 후 상태:
 - image_newline: cuda:7
📦 Model 서브모듈 디바이스 확인:
✅ : cuda:7
✅ vision_model: cuda:7
✅ vision_model.vision_model: cuda:7
✅ vision_model.vision_model.embeddings: cuda:7
✅ vision_model.vision_model.embeddings.patch_embedding: cuda:7
✅ vision_model.vision_model.embeddings.position_embedding: cuda:7
✅ vision_model.vision_model.encoder: cuda:7
✅ vision_model.vision_model.encoder.layers: cuda:7
✅ vision_model.vision_model.encoder.layers.0: cuda:7
✅ vision_model.vision_model.encoder.layers.0.layer_norm1: cuda:7
✅ vision_model.vision_model.encoder.layers.0.self_attn: cuda:7
✅ vision_model.vision_model.encoder.layers.0.self_attn.k_proj: cuda:7
✅ vision_model.vision_model.encoder.layers.0.self_attn.v_proj: cuda:7
✅ vision_model.vision_model.encoder.layers.0.self_attn.q_proj: cuda:7
✅ vision_model.vision_model.encoder.layers.0.self_attn.out_proj: cuda:7
✅ vision_model.vision_model.encoder.layers.0.layer_norm2: cuda:7
✅ vision_model.vision_model.encoder.layers.0.mlp: cuda:

Evaluating:   2%|▏         | 49/2400 [00:32<24:45,  1.58it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
INFO:__main__:✅ Progress: 50/2400 | Accuracy: 0.7200
Evaluating:   4%|▍         | 99/2400 [01:02<26:36,  1.44it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
INFO:__main__:✅ Progress: 100/2400 | Accuracy: 0.7900
Evaluating:   6%|▌         | 149/2400 [01:35<17:46,  2.11it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
INFO:__main__:✅ Progress: 150/2400 | Accuracy: 0.8400
Evaluating:   8%|▊         | 199/2400 [02:05<25:12,  1.45it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for


=== Test Results ===
Accuracy: 0.4742
Precision: 0.6347
Recall: 0.4742
F1 Score: 0.4726

=== Per-Class Performance ===
행복: P=0.757, R=0.708, F1=0.731, Support=400.0
슬픔: P=0.571, R=0.463, F1=0.511, Support=400.0
분노: P=0.714, R=0.468, F1=0.565, Support=400.0
공포: P=0.764, R=0.138, F1=0.233, Support=400.0
놀람: P=0.729, R=0.263, F1=0.386, Support=400.0
중립: P=0.274, R=0.807, F1=0.409, Support=400.0


### 각 라벨 별 예측 빈도

In [ ]:
# predict.csv 파일 로드
df = pd.read_csv("/home/jieun/workspace/clovax/finetune/sft/test_results/predictions.csv")  # 파일 경로 필요 시 수정

# true_label별로 predicted_label 카운트
grouped_counts = df.groupby("true_label")["predicted_label"].value_counts().unstack(fill_value=0)

# 결과 확인
print(grouped_counts)

# 저장하고 싶다면:
grouped_counts.to_csv("label_prediction_counts.csv", encoding="utf-8-sig")


predicted_label  unknown  공포   놀람   분노   슬픔   중립   행복
true_label                                           
공포                     4  55    8    5  115  202   11
놀람                    14   4  105   16    3  233   25
분노                     6   8    3  187    7  176   13
슬픔                     2   4   14   32  185  151   12
중립                    18   1    7   15    6  323   30
행복                     1   0    7    7    8   94  283


: 

## ======================================================================================

### SFT Model test code (naver-hyperclovax/HyperCLOVAX-SEED-Vision-Instruct-3B) / Claude